In [ ]:

from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id="openai.gpt-oss-120b-1:0",
    # model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    # model_id="us.meta.llama3-3-70b-instruct-v1:0",
    model_kwargs=dict(temperature=0.2),
    region="us-west-2"
)

# Using Checkpoint for Chat History

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import InMemorySaver


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}


graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)

# Create an in-memory checkpointer
memory = InMemorySaver()

graph = graph_builder.compile(checkpointer=memory)

thread_id = "user_session_1"
config = {"configurable": {"thread_id": thread_id}}


def stream_graph_updates(user_input: str):
    for event in graph.stream(
            {"messages": [{"role": "user", "content": user_input}]}, config=config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

In [ ]:
stream_graph_updates("Hello! My name is Dan.")

Assistant: <reasoning>The user says "Hello! My name is Dan." Likely they want a greeting. We respond friendly. Ask how can help.</reasoning>Hi Dan! 👋 Nice to meet you. How can I help you today?


In [ ]:
stream_graph_updates("I am a Virgo.")

Assistant: <reasoning>The user says "I am a Virgo." Probably they want to talk about zodiac sign. We can respond acknowledging and maybe ask about traits or what they'd like to discuss. Keep friendly.</reasoning>Got it—Virgo it is! ♍️  
Virgos are often described as detail‑oriented, practical, and analytical. Do you feel those traits describe you, or is there something specific about being a Virgo you’d like to chat about?


In [ ]:
stream_graph_updates("Guess which month I was born?")

Assistant: <reasoning>User: "Guess which month I was born?" They said they are a Virgo. Virgo dates are August 23 – September 22. So month could be August or September. Could guess one. Maybe ask if August or September. Provide guess.</reasoning>Since you mentioned you’re a Virgo (August 23 – September 22), I’ll guess you were born in **September**. 🌟 Was I close?


In [ ]:
stream_graph_updates("Did you get my name?")

Assistant: <reasoning>User asks "Did you get my name?" We previously responded that name is Dan. So answer confirming we have it. Possibly ask if they'd like to be called something else.</reasoning>Yes—I’ve got your name as **Dan**. 😊 Let me know if there’s anything else you’d like to talk about!


# Using List for Chat History

In [ ]:
from langgraph.graph import StateGraph, START, END

from langgraph.graph import StateGraph, START, END
from typing import TypedDict


class State(TypedDict):
    history: str  # conversation history as plain text
    user_input: str
    response: str


graph = StateGraph(State)


def generate_response(state: State):
    # Build prompt by combining history and user input
    prompt = f"{state['history']}\nUser: {state['user_input']}\nAssistant:"
    # Call the non-chat LLM with the prompt string
    completion = llm.invoke(prompt)
    # Append assistant response to history
    new_history = f"{prompt} {completion}"
    return {"response": completion, "history": new_history}


graph.add_node("generate_response", generate_response)
graph.add_edge(START, "generate_response")
graph.add_edge("generate_response", END)

app = graph.compile()


In [ ]:

state = {"history": "", "user_input": "", "response": ""}


In [ ]:
def chat(user_input):
    state["user_input"] = user_input
    result = app.invoke(state)
    print("Assistant:", result["response"])
    state["history"] = result["history"]

In [ ]:
chat("Hello!")

Assistant: content='<reasoning>The user says "Hello!" again. We should respond politely. Probably greet back and ask how can help.</reasoning>Hello! 👋 How can I help you today?' additional_kwargs={'usage': {'prompt_tokens': 371, 'completion_tokens': 42, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 413}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} response_metadata={'usage': {'prompt_tokens': 371, 'completion_tokens': 42, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 413}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} id='run--ca9006b9-2594-4667-95d1-e1c3b93c72db-0' usage_metadata={'input_tokens': 371, 'output_tokens': 42, 'total_tokens': 413, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [ ]:
chat("My name is Dan")

Assistant: content='<reasoning>The user says "My name is Dan". We should respond acknowledging and perhaps ask how we can help. Keep friendly tone.</reasoning>Nice to meet you, Dan! How can I assist you today?' additional_kwargs={'usage': {'prompt_tokens': 676, 'completion_tokens': 48, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 724}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} response_metadata={'usage': {'prompt_tokens': 676, 'completion_tokens': 48, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 724}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} id='run--70e73097-3d86-455b-ad9a-505e92f2b4cd-0' usage_metadata={'input_tokens': 676, 'output_tokens': 48, 'total_tokens': 724, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}


In [ ]:
chat("I am a virgo")

Assistant: content='<reasoning>We need to respond to "I am a virgo". The user gave astrological sign. Could respond with something about Virgo traits, ask about interest, or continue conversation. Keep friendly. Possibly ask if they want horoscope or personality traits. Should be concise.</reasoning>Nice to meet you, Dan! 🌟 As a Virgo, you’re probably detail‑oriented, practical, and love a good sense of order. Does that sound about right? Let me know if you’d like to chat about anything—maybe a daily horoscope, some Virgo‑focused tips, or something completely different!' additional_kwargs={'usage': {'prompt_tokens': 988, 'completion_tokens': 126, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1114}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} response_metadata={'usage': {'prompt_tokens': 988, 'completion_tokens': 126, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total

In [ ]:
chat("Say my name")

Assistant: content='<reasoning>The user says "Say my name". They previously said their name is Dan. So we should respond by saying "Your name is Dan." Possibly ask if they\'d like anything else. Keep friendly.</reasoning>Your name is Dan. 😊 Let me know if there’s anything else you’d like to talk about!' additional_kwargs={'usage': {'prompt_tokens': 1385, 'completion_tokens': 69, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1454}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} response_metadata={'usage': {'prompt_tokens': 1385, 'completion_tokens': 69, 'cache_read_input_tokens': 0, 'cache_write_input_tokens': 0, 'total_tokens': 1454}, 'stop_reason': None, 'thinking': {}, 'model_id': 'openai.gpt-oss-120b-1:0', 'model_name': 'openai.gpt-oss-120b-1:0'} id='run--fbb9c612-543c-4653-83fd-4b6ffd757d56-0' usage_metadata={'input_tokens': 1385, 'output_tokens': 69, 'total_tokens': 1454, 'input_to